# Project - Natural Language Processing (NLP)

This Project aimes to:

1- Collect data from the Brazilian Monetary Policy Committee (COPOM).

2- Based on this data, predict the most probable trajectory outcome from the next meeting, regarding the basic interest rate of the Central Bank of Brazil (SELIC rate).

## Importing Libraries

In [1]:
import pandas as pd
pd.set_option("display.max_colwidth", 150)
pd.set_option("display.min_rows", 20)

import matplotlib
matplotlib.rcParams["figure.figsize"] = (36,16)

import urllib.request, json

from tqdm import tqdm

from bs4 import BeautifulSoup
import re

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
import string
import pdfplumber
import fitz  # PyMuPDF

import os

import matplotlib.pyplot as plt

# 1. Collecting and processing Data / Creating Variables

Opening .txt files

In [2]:
# Obtém o caminho do diretório atual
current_directory = os.getcwd()

# Lista todos os arquivos no diretório atual
all_files = os.listdir(current_directory)

# Filtra apenas os arquivos com extensão .txt
file_list = [file for file in all_files if file.endswith(".txt")]

# Exibe a lista de nomes de arquivos .txt
print(file_list)


['202.txt', '203.txt', '204.txt', '205.txt', '206.txt', '207.txt', '208.txt', '209.txt', '210.txt', '211.txt', '212.txt', '213.txt', '214.txt', '215.txt', '216.txt', '217.txt', '218.txt', '219.txt', '220.txt', '221.txt', '222.txt', '223.txt', '224.txt', '225.txt', '226.txt', '227.txt', '228.txt', '229.txt', '230.txt', '231.txt', '232.txt', '233.txt', '234.txt', '235.txt', '236.txt', '237.txt', '238.txt', '239.txt', '240.txt', '241.txt', '242.txt', '243.txt', '244.txt', '245.txt', '246.txt', '247.txt', '248.txt', '249.txt', '250.txt', '251.txt', '252.txt', '253.txt', '254.txt', '255.txt', '256.txt', '257.txt', '258.txt', '259.txt', '260.txt']


Using stopwords in the language of the text (portuguese).

In [3]:
# Load stopwords
stopwords = nltk.corpus.stopwords.words('portuguese')

In [4]:
# Additional list of words to be removed
ad_list = ['meses', 'ano', 'mês', 'relação', 'doze', 'período', 'sobre', 'janeiro', 'fevereiro', 'março', 'abril',
           'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro', 'nível', '-', 'bilhões',
           'anterior', 'copom', 'banco', 'central', 'após', 'desde', 'comparação', 'respectivamente', 'anterior',
           'monetária', 'índice', 'ante', 'pp', 'variação', 'reunião', 'trimestre', 'comitê', 'chefe', 'segue',
           'departamento', 'membros', 'ainda', 'cenário', '2022', '2023', 'a.a.', '2023.', 'básico']

In [5]:
stopwords = set(list(stopwords) + list(string.punctuation) + ad_list)

In [6]:
dict_top = {}

for arquivo in file_list:
    with open(arquivo.replace('.pdf', '.txt'),'r', encoding='utf-8') as f:
        # Remover caracteres não alfanuméricos
        texto = f.read().replace(r'[^\w\s\-áéíóúãõâêîôûàèìòùäëïöüç]', '').lower()
        # Substituir quebras de linha por espaços
        texto = texto.replace('\n', ' ')
        frequency_distribution = FreqDist(word.lower() for word in word_tokenize(texto) if word not in stopwords)
        dict_top[arquivo.replace('.pdf', '')] = frequency_distribution.most_common(10)

Creating a Dataframe with the 10 most common words in each minute.

In [7]:
df_words = pd.DataFrame(dict_top)

In [8]:
df_words = df_words.T

# Obtém os nomes atuais das colunas
nomes_colunas_atuais = df_words.columns.tolist()

# Gera os novos nomes de colunas com o prefixo "df_words"
novos_nomes_colunas = ['df_words_' + str(coluna) for coluna in nomes_colunas_atuais]

# Renomeia as colunas
df_words = df_words.rename(columns=dict(zip(nomes_colunas_atuais, novos_nomes_colunas)))



Creating a Dataframe with the 10 most common bigrams in each minute.

In [9]:
dict_top_bigram = {}

for arquivo in file_list:
    with open(arquivo.replace('.pdf', '.txt'), 'r', encoding='utf-8') as f:
        # Remover caracteres não alfanuméricos
        texto = f.read().replace(r'[^\w\s\-áéíóúãõâêîôûàèìòùäëïöüç]', '').lower()
        # Substituir quebras de linha por espaços
        texto = texto.replace('\n', ' ')
        frequency_distribution = FreqDist(word for word in nltk.bigrams([word.lower() for word in word_tokenize(texto) if word not in stopwords]))
        dict_top_bigram[arquivo.replace('.pdf', '')] = frequency_distribution.most_common(10)

In [10]:
df_bigram = pd.DataFrame(dict_top_bigram)

In [11]:
df_bigram = df_bigram.T

# Obtém os nomes atuais das colunas
nomes_colunas_atuais = df_bigram.columns.tolist()

# Gera os novos nomes de colunas com o prefixo "df_words"
novos_nomes_colunas = ['df_bigram_' + str(coluna) for coluna in nomes_colunas_atuais]

# Renomeia as colunas
df_bigram = df_bigram.rename(columns=dict(zip(nomes_colunas_atuais, novos_nomes_colunas)))

Creating a Dataframe with the 10 most common trigrams in each minute.

In [12]:
dict_top_trigram = {}

for arquivo in file_list:
    with open(arquivo.replace('.pdf', '.txt'), 'r', encoding='utf-8') as f:
        # Remover caracteres não alfanuméricos
        texto = f.read().replace(r'[^\w\s\-áéíóúãõâêîôûàèìòùäëïöüç]', '').lower()
        # Substituir quebras de linha por espaços
        texto = texto.replace('\n', ' ')
        frequency_distribution = FreqDist(word for word in nltk.trigrams([word.lower() for word in word_tokenize(texto) if word not in stopwords]))
        dict_top_trigram[arquivo.replace('.pdf', '')] = frequency_distribution.most_common(10)

In [13]:
df_trigram = pd.DataFrame(dict_top_trigram)

In [14]:
df_trigram = df_trigram.T

# Obtém os nomes atuais das colunas
nomes_colunas_atuais = df_trigram.columns.tolist()

# Gera os novos nomes de colunas com o prefixo "df_words"
novos_nomes_colunas = ['df_trigram_' + str(coluna) for coluna in nomes_colunas_atuais]

# Renomeia as colunas
df_trigram = df_trigram.rename(columns=dict(zip(nomes_colunas_atuais, novos_nomes_colunas)))

Collecting our target: the communication expressing the decision of the Central Bank of Brazil regarding the Selic Interest Rate (Taxa Selic).

In [15]:
# Using 1000 to ensure all available data is being collected
url_alvo = 'https://www.bcb.gov.br/api/servico/sitebcb/copom/comunicados?quantidade=1000'

In [16]:
with urllib.request.urlopen(url_alvo) as url:
    data = json.load(url)
    print(data)

{'conteudo': [{'nro_reuniao': 260, 'dataReferencia': '2024-01-31', 'titulo': '260ª reunião - Copom reduz a taxa Selic para 11,25% a.a.'}, {'nro_reuniao': 259, 'dataReferencia': '2023-12-13', 'titulo': '259ª reunião - Copom reduz a taxa Selic para 11,75% a.a.'}, {'nro_reuniao': 258, 'dataReferencia': '2023-11-01', 'titulo': '258ª reunião - Copom reduz a taxa Selic para 12,25% a.a.'}, {'nro_reuniao': 257, 'dataReferencia': '2023-09-20', 'titulo': '257ª reunião - Copom reduz a taxa Selic para 12,75% a.a.'}, {'nro_reuniao': 256, 'dataReferencia': '2023-08-02', 'titulo': '256ª reunião - Copom reduz a taxa Selic para 13,25% a.a.'}, {'nro_reuniao': 255, 'dataReferencia': '2023-06-21', 'titulo': '255ª reunião - Copom mantém a taxa Selic em 13,75% a.a.'}, {'nro_reuniao': 254, 'dataReferencia': '2023-05-03', 'titulo': '254ª reunião - Copom mantém a taxa Selic em 13,75% a.a.'}, {'nro_reuniao': 253, 'dataReferencia': '2023-03-22', 'titulo': '253ª reunião - Copom mantém a taxa Selic em 13,75% a.a.'

In [17]:
df_alvo = pd.DataFrame(data['conteudo'])

In [18]:
index_202 = df_alvo[df_alvo['nro_reuniao'] == 202].index[0]
df_alvo = df_alvo.loc[:index_202]

Simplifying the output filtering only if the interst rate : reduces, maintains or increases

In [19]:
# Assuming df_alvo is your DataFrame
df_alvo['alvo'] = df_alvo['titulo'].apply(lambda x: 'reduz' if 'reduz' in x else ('mantém' if 'mantém' in x else 'eleva'))

Merging all data in one Dataframe

In [20]:
df_words.index = df_words.index.str.rstrip('.txt')
df_bigram.index = df_bigram.index.str.rstrip('.txt')
df_trigram.index = df_trigram.index.str.rstrip('.txt')


In [21]:
# Converta o índice de df_words para o mesmo tipo da coluna 'nro_reuniao' de df_alvo
df_words.index = df_words.index.astype(df_alvo['nro_reuniao'].dtype)
df_bigram.index = df_bigram.index.astype(df_alvo['nro_reuniao'].dtype)
df_trigram.index = df_trigram.index.astype(df_alvo['nro_reuniao'].dtype)

# Realize o inner join
df = pd.merge(df_alvo, df_words, left_on='nro_reuniao', right_index=True, how='inner')
df = pd.merge(df, df_bigram, left_on='nro_reuniao', right_index=True, how='inner')
df = pd.merge(df, df_trigram, left_on='nro_reuniao', right_index=True, how='inner')

In [22]:
df

,nro_reuniao,dataReferencia,titulo,alvo,df_words_0,df_words_1,df_words_2,df_words_3,df_words_4,df_words_5,...,df_trigram_0,df_trigram_1,df_trigram_2,df_trigram_3,df_trigram_4,df_trigram_5,df_trigram_6,df_trigram_7,df_trigram_8,df_trigram_9
0,260,2024-01-31,"260ª reunião - Copom reduz a taxa Selic para 11,25% a.a.",reduz,"(inflação, 38)","(política, 22)","(dinâmica, 17)","(expectativas, 13)","(mercado, 12)","(atividade, 11)",...,"((260ª, 30, 31), 4)","((0,50, ponto, percentual), 4)","((manter, política, contracionista), 3)","((debate, início, processo), 2)","((início, processo, flexibilização), 2)","((processo, flexibilização, política), 2)","((flexibilização, política, principais), 2)","((política, principais, economias), 2)","((crescimento, econômico, resiliência), 2)","((econômico, resiliência, consumo), 2)"
1,259,2023-12-13,"259ª reunião - Copom reduz a taxa Selic para 11,75% a.a.",reduz,"(inflação, 34)","(política, 22)","(expectativas, 13)","(dinâmica, 11)","(atividade, 10)","(mercado, 10)",...,"((259ª, 12, 13), 4)","((0,50, ponto, percentual), 4)","((manter, política, contracionista), 3)","((resiliência, consumo, famílias), 2)","((hiato, produto, apertado), 2)","((cautela, condução, política), 2)","((discussão, condução, política), 2)","((serenidade, moderação, condução), 2)","((moderação, condução, política), 2)","((ritmo, apropriado, manter), 2)"
2,258,2023-11-01,"258ª reunião - Copom reduz a taxa Selic para 12,25% a.a.",reduz,"(inflação, 40)","(política, 21)","(atividade, 14)","(maior, 14)","(juros, 13)","(expectativas, 13)",...,"((258ª, 31, 1º), 5)","((0,50, ponto, percentual), 4)","((elevação, taxas, juros), 3)","((hiato, produto, apertado), 3)","((manter, política, contracionista), 3)","((mercado, trabalho, aquecido), 2)","((taxas, juros, longo), 2)","((juros, longo, prazo), 2)","((múltiplos, canais, transmissão), 2)","((taxa, câmbio, preço), 2)"
3,257,2023-09-20,"257ª reunião - Copom reduz a taxa Selic para 12,75% a.a.",reduz,"(inflação, 48)","(política, 21)","(dinâmica, 16)","(expectativas, 15)","(crescimento, 14)","(atividade, 14)",...,"((257ª, 19, 20), 5)","((hiato, produto, apertado), 4)","((0,50, ponto, percentual), 4)","((resiliência, atividade, econômica), 3)","((manter, política, contracionista), 3)","((convergência, inflação, meta), 3)","((ciclos, aperto, monetário), 2)","((elevação, taxas, juros), 2)","((taxas, juros, longo), 2)","((juros, longo, prazo), 2)"
4,256,2023-08-02,"256ª reunião - Copom reduz a taxa Selic para 13,25% a.a.",reduz,"(inflação, 58)","(política, 21)","(expectativas, 21)","(dinâmica, 19)","(meta, 16)","(maior, 15)",...,"((256ª, 1º, 2), 6)","((0,50, ponto, percentual), 6)","((convergência, inflação, meta), 5)","((manter, política, contracionista), 4)","((hiato, produto, apertado), 3)","((desaceleração, gradual, atividade), 2)","((acima, meta, inflação), 2)","((estágio, processo, desinflacionário), 2)","((acima, patamar, compatível), 2)","((patamar, compatível, meta), 2)"
5,255,2023-06-21,"255ª reunião - Copom mantém a taxa Selic em 13,75% a.a.",mantém,"(inflação, 44)","(expectativas, 23)","(processo, 20)","(política, 20)","(taxa, 15)","(maior, 14)",...,"((255ª, 20, 21), 5)","((pib, referente, primeiro), 2)","((setores, cíclicos, economia), 2)","((componentes, sensíveis, ciclo), 2)","((sensíveis, ciclo, econômico), 2)","((elevação, taxa, neutra), 2)","((contribui, processo, desinflacionário), 2)","((compatível, atual, estágio), 2)","((atual, estágio, ciclo), 2)","((estágio, ciclo, política), 2)"
6,254,2023-05-03,"254ª reunião - Copom mantém a taxa Selic em 13,75% a.a.",mantém,"(inflação, 47)","(política, 21)","(expectativas, 20)","(processo, 15)","(maior, 13)","(taxa, 13)",...,"((254ª, 2, 3), 5)","((projeções, inflação, situam-se), 2)","((taxa, juros, neutra), 2)","((controle, inflação, através), 2)","((convergência, inflação, metas), 2)","((efeitos, medidas, tributárias), 2)","((expectativas, inflação, seguem), 2)","((inflação, seguem, desancoradas), 2)","((conselho, monetário, naciona

Cleaning Dataframe

In [23]:
# List of columns to drop
columns_to_drop = ['nro_reuniao', 'dataReferencia', 'titulo']

# Separate features (X) and target variable (y)
df_ml = df.drop(columns=columns_to_drop, axis=1)

Shifting the target by 1 cell. 

The objective here is to predict the next output based on the current minute text. Therefore, we need the last minute to predict the next decision.

In [24]:
df_ml['alvo'] = df_ml['alvo'].shift(1)

Making copies of the DataFrame for further operations

In [25]:
df_pred = df_ml.copy()

Converting the Target into 0,1 or 2 values

# 2. Applying Machine Learning Models to predict the Output of the Brazil Central Bank decision

In [26]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import pickle

# Carregar o modelo treinado
with open('model_Selic_Rate.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Carregar o vetorizador TF-IDF usado durante o treinamento
with open('vectorizer_Selic_Rate.pkl', 'rb') as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)

# Concatenação dos dados textuais
df_text = df_pred.apply(lambda row: ' '.join(map(str, row)), axis=1)

# Vetorização do texto usando TF-IDF
X_df = vectorizer.transform(df_text)

# Fazer previsões para todo o DataFrame
predictions_df = model.predict(X_df)
probabilities_df = model.predict_proba(X_df)


# Adicionando as previsões ao DataFrame original
df_pred['predictions'] = predictions_df
# Adicionando as probabilidades ao DataFrame original para cada classe
for i, class_label in enumerate(model.classes_):
    col_name = f'probability_{class_label}'
    df_pred[col_name] = probabilities_df[:, i]

# Exibindo o DataFrame resultante
df_pred

,alvo,df_words_0,df_words_1,df_words_2,df_words_3,df_words_4,df_words_5,df_words_6,df_words_7,df_words_8,...,df_trigram_4,df_trigram_5,df_trigram_6,df_trigram_7,df_trigram_8,df_trigram_9,predictions,probability_0,probability_1,probability_2
0,NaN,"(inflação, 38)","(política, 22)","(dinâmica, 17)","(expectativas, 13)","(mercado, 12)","(atividade, 11)","(trabalho, 10)","(maior, 10)","(econômica, 9)",...,"((início, processo, flexibilização), 2)","((processo, flexibilização, política), 2)","((flexibilização, política, principais), 2)","((política, principais, economias), 2)","((crescimento, econômico, resiliência), 2)","((econômico, resiliência, consumo), 2)",0,0.690,0.180,0.130
1,reduz,"(inflação, 34)","(política, 22)","(expectativas, 13)","(dinâmica, 11)","(atividade, 10)","(mercado, 10)","(econômica, 9)","(trabalho, 9)","(moderação, 7)",...,"((hiato, produto, apertado), 2)","((cautela, condução, política), 2)","((discussão, condução, política), 2)","((serenidade, moderação, condução), 2)","((moderação, condução, política), 2)","((ritmo, apropriado, manter), 2)",0,0.735,0.185,0.080
2,reduz,"(inflação, 40)","(política, 21)","(atividade, 14)","(maior, 14)","(juros, 13)","(expectativas, 13)","(taxa, 13)","(econômica, 12)","(riscos, 12)",...,"((manter, política, contracionista), 3)","((mercado, trabalho, aquecido), 2)","((taxas, juros, longo), 2)","((juros, longo, prazo), 2)","((múltiplos, canais, transmissão), 2)","((taxa, câmbio, preço), 2)",0,0.810,0.125,0.065
3,reduz,"(inflação, 48)","(política, 21)","(dinâmica, 16)","(expectativas, 15)","(crescimento, 14)","(atividade, 14)","(hiato, 14)","(produto, 14)","(maior, 13)",...,"((manter, política, contracionista), 3)","((convergência, inflação, meta), 3)","((ciclos, aperto, monetário), 2)","((elevação, taxas, juros), 2)","((taxas, juros, longo), 2)","((juros, longo, prazo), 2)",0,0.870,0.085,0.045
4,reduz,"(inflação, 58)","(política, 21)","(expectativas, 21)","(dinâmica, 19)","(meta, 16)","(maior, 15)","(hiato, 15)","(taxa, 14)","(serviços, 13)",...,"((hiato, produto, apertado), 3)","((desaceleração, gradual, atividade), 2)","((acima, meta, inflação), 2)","((estágio, processo, desinflacionário), 2)","((acima, patamar, compatível), 2)","((patamar, compatível, meta), 2)",0,0.490,0.315,0.195
5,reduz,"(inflação, 44)","(expectativas, 23)","(processo, 20)","(política, 20)","(taxa, 15)","(maior, 14)","(preços, 13)","(avalia, 10)","(metas, 9)",...,"((sensíveis, ciclo, econômico), 2)","((elevação, taxa, neutra), 2)","((contribui, processo, desinflacionário), 2)","((compatível, atual, estágio), 2)","((atual, estágio, ciclo), 2)","((estágio, ciclo, política), 2)",0,0.710,0.220,0.070
6,mantém,"(inflação, 47)","(política, 21)","(expectativas, 20)","(processo, 15)","(maior, 13)","(taxa, 13)","(juros, 12)","(preços, 11)","(metas, 10)",...,"((convergência, inflação, metas), 2)","((efeitos, medidas, tributárias), 2)","((expectativas, inflação, seguem), 2)","((inflação, seguem, desancoradas), 2)","((conselho, monetário, nacional), 2)","((compatível, atual, estágio), 2)",1,0.405,0.440,0.155
7,mantém,"(inflação, 49)","(política, 26)","(expectativas, 22)","(processo, 18)","(maior, 16)","(juros, 15)","(taxa, 15)","(desaceleração, 12)","(metas, 12)",...,"((taxa, básica, juros), 3)","((inflação, globais, mantêm), 2)","((globais, mantêm, resilientes), 2)","((aperto, condições, financeiras), 2)","((sistema, bancário, economias), 2)","((bancário, economias, centrais), 2)",0,0.485,0.405,0.110
8,mantém,"(inflação, 42)","(política, 24)","(expectativas, 19)","(projeções, 16)","(desaceleração, 15)","(juros, 14)","(riscos, 13)","(processo, 12)","(taxa, 12)",...,"((política, combate, covid), 2)","((combate, covid, china), 2)","((covid, china, inverno), 2)","((china, inverno, ameno), 2)","((inverno, ameno, europa), 2)","((aperto, condições, financeiras), 2)",1,0.225,0.740,0.035
9,mantém,"(inflação, 37)","(política, 17)","(expectativas, 13)","(preços, 12)","(riscos, 12)","(juros, 11)","(crescimento, 10)","(maior, 10)",

In [27]:
# Codificação do alvo
class_mapping = {0 : 'reduces', 1 : 'maintains', 2 : 'increases'}
df_pred['predictions'] = df_pred['predictions'].map(class_mapping)

In [28]:
df_pred.rename(columns={'probability_0': 'reduces', 'probability_1': 'maintains', 'probability_2': 'increases'}, inplace=True)

# 3. Showing Prediction for Next Meeting

In [29]:
print('-----------------------------------------------------------------------------------------')
print(f"Based on the text of the last minute from Brazil's Central Bank, minute number "+str(df['nro_reuniao'][0]))
print(f"the next resolution, concering the Brazilian interest ratio, will be:")
print(f' "' + str(df_pred['predictions'][0]) + '" ')
print('-----------------------------------------------------------------------------------------')
print(f'the probabilities of the predictions are:')
print(df_pred[['reduces','maintains','increases']].iloc[0])
print('-----------------------------------------------------------------------------------------')

-----------------------------------------------------------------------------------------
Based on the text of the last minute from Brazil's Central Bank, minute number 260
the next resolution, concering the Brazilian interest ratio, will be:
 "reduces" 
-----------------------------------------------------------------------------------------
the probabilities of the predictions are:
reduces      0.69
maintains    0.18
increases    0.13
Name: 0, dtype: float64
-----------------------------------------------------------------------------------------
